# Example of BOMD MD (NVE, NVT)

In [10]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import Molecular_Dynamics_Basic, Molecular_Dynamics_Langevin
from seqm.seqm_functions.read_xyz import read_xyz

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [14]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Basic'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   303.73   1.570421e-01 -1.363987e+00 -1.206945e+00 ||    258.65   1.337346e-01 1.415662e+00 1.549397e+00 || 
     2   296.18   1.531350e-01 -1.360048e+00 -1.206913e+00 ||    311.06   1.608290e-01 1.388264e+00 1.549093e+00 || 
     3   278.40   1.439427e-01 -1.350787e+00 -1.206844e+00 ||    374.59   1.936758e-01 1.355031e+00 1.548706e+00 || 
     4   252.61   1.306074e-01 -1.337358e+00 -1.206750e+00 ||    438.22   2.265798e-01 1.321721e+00 1.548301e+00 || 
     5   221.76   1.146604e-01 -1.321310e+00 -1.206649e+00 ||    490.66   2.536927e-01 1.294263e+00 1.547956e+00 || 
     6   189.06   9.775012e-02 -1.304309e+00 -1.206559e+00 ||    522.62   2.702134e-01 1.277533e+00 1.547747e+00 || 
     7   157.38   8.137028e-02 -1.287863e+00 -1.206493e+00 ||    529.01   2.735179e-01 1.274206e+00 1.547724e+00 || 
     8   128.91   6.665340e-02 -1.273109e+00 -1.206456e+00 ||    510.29   2.638427e-01 1.284040e+00 1.547882e+00 || 
     9   

## *MD_BOMD_BASIC.0.xyz* and *MD_BOMD_BASIC.1.xyz* files with general info for each step are created. Coordinates, velocities, forces and charges are recorded in the following sequence: x y z vx vy vz fx fy fz q.

## *MD_BOMD_BASIC.Info.0.txt* and *MD_BOMD_BASIC.Info.1.txt* contain additional info for each step (orbital energies, dipole).

## Scale velocities to adjust kinetic energy and compenstate the energy shift

In [15]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Basic_Ectrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, control_energy_shift=True, remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   356.37   1.842588e-01 -1.286218e-01 5.563700e-02 ||    488.19   2.524161e-01 2.128525e+00 2.380941e+00 || 
     2   459.82   2.377441e-01 -1.821071e-01 5.563700e-02 ||    591.05   3.055948e-01 2.075347e+00 2.380941e+00 || 
     3   597.46   3.089099e-01 -2.532729e-01 5.563700e-02 ||    749.68   3.876151e-01 1.993326e+00 2.380941e+00 || 
     4   773.37   3.998610e-01 -3.442240e-01 5.563700e-02 ||    948.07   4.901911e-01 1.890750e+00 2.380941e+00 || 
     5   992.97   5.134036e-01 -4.577666e-01 5.563700e-02 ||   1167.46   6.036230e-01 1.777318e+00 2.380941e+00 || 
     6  1258.54   6.507166e-01 -5.950796e-01 5.563700e-02 ||   1387.43   7.173591e-01 1.663582e+00 2.380941e+00 || 
     7  1563.94   8.086221e-01 -7.529851e-01 5.563700e-02 ||   1586.90   8.204886e-01 1.560453e+00 2.380941e+00 || 
     8  1890.80   9.776215e-01 -9.219845e-01 5.563700e-02 ||   1744.60   9.020262e-01 1.478915e+00 2.380941e+00 || 
     9  2208.27  

## Scale velocities to control temperature

In [16]:
%%time
species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 1.40,  0.0,  0.0],
                   [ 0.00,  0.0,  0.0],
                   [-0.60,  0.94, 0.0],
                   [-0.60, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.40,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/MD_BOMD_Basic_Tctrl'}
molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

md =  Molecular_Dynamics_Basic(seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, scale_vel=[1,400], remove_com=[True, 1], Info_log=True)

Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   400.00   2.068160e-01 -1.770809e-01 2.973512e-02 ||    400.00   2.068160e-01 2.067105e+00 2.273921e+00 || 
     2   400.00   2.068160e-01 -2.701705e-01 -6.335446e-02 ||    400.00   2.068160e-01 1.977663e+00 2.184479e+00 || 
     3   400.00   2.068160e-01 -3.698711e-01 -1.630551e-01 ||    400.00   2.068160e-01 1.875273e+00 2.082089e+00 || 
     4   400.00   2.068160e-01 -4.734252e-01 -2.666092e-01 ||    400.00   2.068160e-01 1.768589e+00 1.975405e+00 || 
     5   400.00   2.068160e-01 -5.780569e-01 -3.712409e-01 ||    400.00   2.068160e-01 1.663527e+00 1.870343e+00 || 
     6   400.00   2.068160e-01 -6.812309e-01 -4.744149e-01 ||    400.00   2.068160e-01 1.563993e+00 1.770809e+00 || 
     7   400.00   2.068160e-01 -7.807953e-01 -5.739793e-01 ||    400.00   2.068160e-01 1.472626e+00 1.679442e+00 || 
     8   400.00   2.068160e-01 -8.750156e-01 -6.681996e-01 ||    400.00   2.068160e-01 1.391334e+00 1.598150e+00 || 
     9   4